In [1]:
#https://scrapfly.io/blog/web-scraping-with-selenium-and-python/

TODO
- явно есть ошибки не корретного парсинга, можно найти кейсы по дешевым билетам
- сделать через распаралеривание (4-6 потоков)
- подумать может сделать без селениума
- на сервер запустить скрипт
- продумать как парсить билеты туда назад (тупо с двумя длинными списками будет некорректно)
- сохранять исходники страниц (или скрины)

In [2]:
from bs4 import BeautifulSoup
import asyncio

In [3]:
from parsel import Selector
from itertools import product

from tqdm import tqdm_notebook as tqdm

from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

service = Service()



In [4]:
def get_page(link):

    driver.get(link)
    time.sleep(20)

    try:
        element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='ui-datepicker-div']//a[@class='ui-state-default']")))
    except TimeoutException:
        pass
    
    page = driver.page_source
    return page

In [32]:
def get_ticket(page):
    soup = BeautifulSoup(page, 'lxml')

    prices = soup.find_all('div', attrs={'class':'ticket-desktop__side-container'})

    price_ticket = list(map(lambda x: x.find('span', attrs={'data-test-id':'price'}), prices))
    price_ticket = list(map(lambda x: x.text.replace('\u202f','').replace('\u2009₽',''), price_ticket))
    
    price_luggage = list(map(lambda x: x.find('div', text='Багаж'), prices))
    price_luggage = list(map(lambda x: x.find_next('span').get_text(strip=True) if x else '0',  price_luggage))  
    del_words = ['+\u2009','\u202f', '\u2009₽', '₽']
    for word in del_words:
        price_luggage = list(map(lambda x: x.replace(word,'') if x else '0', price_luggage))
    
    wo_luggage = list(map(lambda x: 1 if x.find('div', text='Без багажа') else 0, prices))

    duration = soup.find_all('div', attrs={'class':'segment-route__duration'})
    duration = list(map(lambda x: x.text.replace('В пути:  ',''), duration))

    transfer = soup.find_all('div', attrs={'class':'segment-route__path'})
    transfer = list(map(lambda x: x.find_all('div', attrs={'class':'segment-route__stop'}), transfer))
    transfer = list(map(lambda x: list(map(lambda y: y['data-iatas'], x)), transfer))

    number_transfer = list(map(lambda x: len(x), transfer))
    transfer = list(map(lambda x: ', '.join(x), transfer))
    transfer = list(map(lambda x: x.replace('\n',"->"), transfer))

    keys = ['price', 'luggage', 'without_luggage', 'duration', 'transfer', 'number_transfer']

    zipped = zip(price_ticket, price_luggage, wo_luggage, duration, transfer, number_transfer)

    data = [dict(zip(keys, values)) for values in zipped]
    return data

In [33]:
def list_dates(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta   
    return range_dates

In [ ]:
origins =  ['BUD', 'TSR', 'OSI', 'PEV', 'SOB', 'DEB', 'ZAG'] #['LED', 'MOW', 'KZN']
destinations = ['HEL']# ['BEG', 'TGD', 'TIA', 'TIV']
range_date = ['2023-10-20', '2023-11-10']
number_ticket = 1

tickets = []
date_start, date_end = range_date[0], range_date[1]
datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')
range_dates = list_dates(datetime_start, datetime_end)

parameters = list(product(origins, destinations, list_dates(datetime_start, datetime_end)))
driver = webdriver.Chrome(options=options, chrome_options=chrome_options, service = service)

for origin, destination, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
    month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
    date = datetime_object.strftime('%Y-%m-%d')
    page = get_page('https://www.aviasales.ru/search/' + origin + day + month + destination + str(number_ticket))
    tickets_by_date = get_ticket(page)   
    tickets_by_date = [dict(item, **{'date':date, 'origin':origin, 'destination':destination}) for item in tickets_by_date]
    tickets += tickets_by_date
    
    
# for destination, origin, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
#     month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
#     date = datetime_object.strftime('%Y-%m-%d')
#     page = get_page('https://www.aviasales.ru/search/' + origin + day + month + destination + str(number_ticket))
#     tickets_by_date = get_ticket(page)   
#     tickets_by_date = [dict(item, **{'date':date, 'origin':origin, 'destination':destination}) for item in tickets_by_date]
#     tickets += tickets_by_date
    

df = pd.DataFrame(tickets)
df.to_csv('tickets.txt', sep = ';', index = False, encoding='utf-8-sig')

parse ticket:   0%|          | 0/154 [00:00<?, ?it/s]